In [1]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import Debugger3
from debugger.tree_node import NodeRef
from evaluator import Evaluator
from tabulate import tabulate
from tqdm import tqdm
import numpy as np
import networkx as nx

In [2]:
q = "data/projs/bench_unstable/base.z3/d_komodo--verified-svc_handlers.gen.dfyImpl___module.__default.va__lemma__svc__map__data__success.smt2"
r = Evaluator(q)
# print(r.status)
# r.collect_garbage()
report = r.build_report()

In [13]:
report.print_insted()

Instantiation (846):
+-----+--------------------+---------------+--------------+--------------+--------+----------+-------------------------------------------------------+
|     | qname              |   trace_count | stabilized   | action       |   time | result   | edit_path                                             |
|-----+--------------------+---------------+--------------+--------------+--------+----------+-------------------------------------------------------|
| 200 | mariposa_qid_64    |        161143 | False        | inst_keep    | 38.548 | unsat    | data/projs/singleton_09f0c73516/base.z3/87994bfe.smt2 |
| 166 | mariposa_qid_8     |         80677 | False        | inst_keep    | 29.483 | unsat    | data/projs/singleton_09f0c73516/base.z3/fed1cf88.smt2 |
| 364 | mariposa_qid_55    |         76346 | False        | inst_keep    | 32.113 | unsat    | data/projs/singleton_09f0c73516/base.z3/a975e5ec.smt2 |
| 274 | mariposa_qid_435   |         73239 | False        | inst_replace 

In [4]:
editor = r.editor
pa = editor.proof

[DEBUG] [edit] proof path: dbg/09f0c73516/proofs/reseed.2991770306885216523.proof 
[DEBUG] [edit] trace path: dbg/09f0c73516/traces/shuffle.10758862226691364161 


cb00d2687799dd0f6aa87cd833c1bf71.pickle
[WARN] [differ] qid constructor_accessor_axiom not found in dbg/09f0c73516/orig.smt2 


In [5]:
mi = r.get_trace_info()
g = mi.get_trace_graph()
ratios = r.get_trace_graph_ratios()
scores = dict()

for qname, ratio in ratios.items():
    scores[qname] = g.aggregate_scores(ratio)

[INFO] [graph] building dbg/09f0c73516/graphs/shuffle.10758862226691364161.txt 


Error parsing line 47822310 (UnexpectedNewline): "[inst-discovered] theory-solvin"


[INFO] [stats] building dbg/09f0c73516/stats/shuffle.10758862226691364161.txt 


Error parsing line 47822310 (UnexpectedNewline): "[inst-discovered] theory-solvin"


[INFO] [graph] building dbg/09f0c73516/graphs/shuffle.10758862226691364161.txt 


Error parsing line 47822310 (UnexpectedNewline): "[inst-discovered] theory-solvin"


[INFO] [stats] building dbg/09f0c73516/stats/shuffle.10758862226691364161.txt 


Error parsing line 47822310 (UnexpectedNewline): "[inst-discovered] theory-solvin"
100%|██████████| 8524/8524 [00:40<00:00, 208.27it/s]
[DEBUG] saving cache at 09f0c73516.ratios 


In [6]:
scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
print(tabulate(scores, headers=["qname", "Score"], tablefmt="fancy_grid"))

╒════════════════════╤══════════════════╕
│ qname              │            Score │
╞════════════════════╪══════════════════╡
│ mariposa_qid_7996  │      1.01719e+06 │
├────────────────────┼──────────────────┤
│ mariposa_qid_131   │ 619816           │
├────────────────────┼──────────────────┤
│ mariposa_qid_8001  │ 572311           │
├────────────────────┼──────────────────┤
│ mariposa_qid_3692  │ 487644           │
├────────────────────┼──────────────────┤
│ mariposa_qid_5612  │ 477626           │
├────────────────────┼──────────────────┤
│ mariposa_qid_10068 │ 425616           │
├────────────────────┼──────────────────┤
│ mariposa_qid_4632  │ 420351           │
├────────────────────┼──────────────────┤
│ mariposa_qid_6558  │ 417429           │
├────────────────────┼──────────────────┤
│ mariposa_qid_6984  │ 393386           │
├────────────────────┼──────────────────┤
│ mariposa_qid_9893  │ 383674           │
├────────────────────┼──────────────────┤
│ mariposa_qid_5494  │ 375250     

In [16]:
actions = editor.get_singleton_actions()
print(len(actions))
print(len(report.tested))
# report.print_tested()

816
398


In [12]:
# editor.get_quant_action("mariposa_qid_7996")
# editor.edit_by_qname("mariposa_qid_7996")
# editor.edit_by_qname("mariposa_qid_8001")
# editor.save("test.smt2")

[INFO] [edit] [erase] mariposa_qid_7996 
[INFO] [edit] [erase] mariposa_qid_8001 
[INFO] [edit] written to test.smt2 


In [11]:
from debugger.tree_node import AppNode

qii = pa.get_inst_info_under_qname("mariposa_qid_2111")

for i in qii.get_all_insts():
    # print(i)
    # print(pa.dump_node(i))
    children = set()
    if not pa.has_node(i):
        node = pa.lookup_node(i)
        assert isinstance(node, AppNode)
        assert node.name == "or"
        for c in node.children:
            children.add(c)
    else:
        children.add(i)

    reachable = set()
    for c in children:
        reachable.add(c)
        for j in nx.descendants(pa, c):
            reachable.add(j)
    
    ref_counts = dict()
    for j in reachable:
        ref_count = pa.in_degree(j)
        ref_counts[j] = ref_count
    
    ref_counts = sorted(ref_counts.items(), key=lambda x: x[1], reverse=True)

    for j, ref_count in ref_counts:
        print(ref_count, pa.dump_node(j))


40 DatatypeTypeType
26 |step#0@@171|
10 (AsyncDisk.InternalStep.HavocConflictingWritesStep_q |step#0@@171|)
10 (not (= (type |step#0@@171|) DatatypeTypeType))
9 (AsyncDisk.InternalStep.ProcessReadFailureStep_q |step#0@@171|)
8 (AsyncDisk.InternalStep.ProcessWriteStep_q |step#0@@171|)
5 (= (type |step#0@@171|) DatatypeTypeType)
5 (AsyncDisk.InternalStep.HavocConflictingWriteReadStep_q |step#0@@171|)
5 (|$IsA#AsyncDisk.InternalStep| |step#0@@171|)
1 (not (|$IsA#AsyncDisk.InternalStep| |step#0@@171|))
1 (type |step#0@@171|)
